In [ ]:
import numpy as np
from utils import *
from student_utils_sp18 import *
import networkx as nx
%matplotlib inline
print("networkx version: {}".format(nx.__version__))

In [ ]:
# reading a file and get graph
lines = read_file("data/sample_input.txt")

num_kingdoms, kingdom_names, start_kingdom, adj_matrix = data_parser(lines)

# map kingdom names to indices
name2index = {}
index2name = {}

count = 1
for name in kingdom_names:
    name2index[name] = count
    index2name[count] = name
    count += 1

G = adjacency_matrix_to_graph(adj_matrix)

G

In [ ]:
# initialize colors for each edge

color = {}

for node in G.nodes():
        color[node] = "white"
        
        
conquering_cost = {}
for node in G.nodes():
    conquering_cost[node] = adj_matrix[node][node]
    
nx.set_node_attributes(G, conquering_cost, "conquering_cost")
nx.set_node_attributes(G, color, "color")
# Example use G.node[1]['color'] gets the color of that node

In [ ]:
# calculate the heuristic for all nodes on the graph

def conquer_gain(G, node):
    """Given a Graph and a node it calculates the value of the heuristic for the node"""
    """conquering_cost_neighbors/conquering_cost_current_node larger is better"""
    
    if G.node[node]['color'] =="black": # do not calculate the heuristic for black nodes
        return 0
    else:
        neighbors = nx.neighbors(G, node)
        
        neighbors_conquering_cost = 0
        for n in neighbors:
            neighbors_conquering_cost += G.node[n]['conquering_cost']
        
        return neighbors_conquering_cost/G.node[node]['conquering_cost']

In [ ]:
# update the colors in the graph

def update_graph(G, node):
    G.node[node]['color'] = "black"
    neighbors = nx.neighbors(G, node)
    
    for n in neighbors:
        if G.node[n]['color'] == "white":
            G.node[n]['color'] = "grey"
    return

In [ ]:
# find the the nodes that we need to visit based on heuristic

def choose_node(G):
    # calculate heuristic for everyone
    heuristic_value = []

    for node in G.nodes():
        heuristic_value.append(conquer_gain(G, node))

    chosen_node = np.argmax(heuristic_value)
    return chosen_node

def all_conquered(G):
    lst = [l[1] for l in nx.get_node_attributes(G, "color").items()]
    
    return not "white" in lst

In [ ]:
# Identify components

nodes_to_visit = []

while not all_conquered(G):
    next_node = choose_node(G)
    nodes_to_visit.append(next_node)
    update_graph(G, next_node)

In [ ]:
# another way to identify components using an approximation to the 
# min weight dominating set

from networkx.algorithms.approximation import *

min_weighted_dominating_set(G, "conquering_cost")

In [ ]:
# Get the Steiner Tree based on the nodes you have identofied
from networkx.algorithms.traversal import *

# GET THE FULL WALK

start_index = name2index[start_kingdom]
ST = steinertree.steiner_tree(G, nodes_to_visit+[start_index])

In [ ]:
# recover the order of traversal of the tree

def find_traversal(tree, start_index):
    """Recovers the traversal of the Steiner tree using DFS (the full walk tho)"""
    
    vertex_order = list(dfs_preorder_nodes(tree, start_index))+[start_index]
    current_node = start_index
    full_walk = [start_index]

    for next_node in vertex_order[1:]:
        #print(current_node, next_node)
        path = list(nx.all_simple_paths(tree, current_node, next_node))[0]
        full_walk.extend(path[1:])
        current_node = next_node
    return full_walk

In [ ]:
traversal = find_traversal(ST, start_index)
conquered = dict(zip(nodes_to_visit, [False]*len(nodes_to_visit)))
to_conquer = []

for n in traversal:
    if n in nodes_to_visit and conquered[n] == False:
        to_conquer.append(n)
        conquered[n]=True
        
print("Traversal: {}".format(traversal))
print("To conquer: {}".format(to_conquer))
ccost = sum(G.node[i]['conquering_cost'] for i in to_conquer)
tcost = sum(G.get_edge_data(traversal[i], traversal[i+1])['weight'] for i in range(len(traversal)-1))
print("Cost of conquering: {}".format(ccost))
print("Cost of travelling: {}".format(tcost))

# Test run with Sue's input 1 file

In [ ]:
# reading a file and get graph
lines = read_file("data/input1.txt")

num_kingdoms, kingdom_names, start_kingdom, adj_matrix = data_parser(lines)

# map kingdom names to indices
name2index = {}
index2name = {}

count = 1
for name in kingdom_names:
    name2index[name] = count
    index2name[count] = name
    count += 1


G = adjacency_matrix_to_graph(adj_matrix)
pos = nx.spring_layout(G)
is_metric(G)

labels = nx.get_edge_attributes(G,'weight')
plt.figure(3,figsize=(8,8)) 
nx.draw_networkx_edge_labels(G,pos,edge_labels=labels)
nx.draw(G, pos, with_labels=True)

In [ ]:
def solver(G, start_index, adj_matrix):
    
    #adj_matrix = nx.adj_matrix(G).todense().tolist()
    
    # set the colors
    color = {}

    for node in G.nodes():
            color[node] = "white"


    conquering_cost = {}
    for node in G.nodes():
        conquering_cost[node] = adj_matrix[node][node]

    nx.set_node_attributes(G, conquering_cost, "conquering_cost")
    nx.set_node_attributes(G, color, "color")
    
    
    # find nodes to visit
    
    nodes_to_visit = []

    while not all_conquered(G):
        next_node = choose_node(G)
        nodes_to_visit.append(next_node)
        update_graph(G, next_node)
    
    # Handle the edge case from below
    if nodes_to_visit[0]==start_index:
        return [start_index], [start_index]
    
    # get Steiner tree
    # CURRENTLY HAS THE BUG THAT THE STEINER TREE DOES NOT WORK WHEN YOU ONLY HAVE 1 TARGET VERTEX
    ST = steinertree.steiner_tree(G, set(nodes_to_visit+[start_index]))

    # Find the traversal based on the Steiner tree
    traversal = find_traversal(ST, start_index)
    conquered = dict(zip(nodes_to_visit, [False]*len(nodes_to_visit)))
    to_conquer = []

    for n in traversal:
        if n in nodes_to_visit and conquered[n] == False:
            to_conquer.append(n)
            conquered[n]=True
            
    return traversal, to_conquer

In [ ]:
solver(G, start_index, adj_matrix)

In [ ]:
for i in range(7):
    print("Conquering cost {}: {}".format(i, G.node[i]['conquering_cost']))

In [ ]:
# print total cost for the tour

tour, conquer = solver(G, start_index, adj_matrix)

def calculate_cost(tour, conquer, G):
    cost = 0
    for i in range(len(tour)-1):
        cost += G.get_edge_data(tour[i], tour[i+1])['weight']

    for i in conquer:
        cost += G.node[i]['conquering_cost']
        
    return cost

calculate_cost(tour, conquer, G)

In [ ]:
tour, conquer

In [ ]:
plt.figure(3,figsize=(8,8)) 
nx.draw_networkx_edge_labels(G,pos,edge_labels=labels)
nx.draw(G, pos, with_labels=True)

## Test with Sona's input

In [ ]:
adjacency_matrix = [[1, 4, 3, 'x'], 
                     [4, 1, 5, 13], 
                     [3, 5, 1, 12], 
                     ['x', 13, 12, 1]]

In [ ]:
G = adjacency_matrix_to_graph(adjacency_matrix)
is_metric(G)

def draw_graph(G):
    pos = nx.spring_layout(G)
    labels = nx.get_edge_attributes(G,'weight')
    plt.figure(3,figsize=(8,8)) 
    nx.draw_networkx_edge_labels(G,pos,edge_labels=labels)
    nx.draw(G, pos, with_labels=True)
    
nx.draw(G)

In [ ]:
tour, conquer = solver(G, 0, adjacency_matrix)

In [ ]:
calculate_cost(tour, conquer, G)

### New input using random graphs

In [ ]:
# idea to generate graph:
# 1. Make a random Erdos Renyi Graph
# 2. Connect fully
# 3. remove edges randomly

In [ ]:
Q = nx.erdos_renyi_graph(10,0.6)
random_weights = np.random.randint(1, 10, len(Q.edges()))
dicti = dict(zip(list(Q.edges()), random_weights))
nx.set_edge_attributes(Q, dicti, "weight")

In [ ]:
# make the fully connected path
# guarantees metric property

for lst in list(nx.shortest_paths.all_pairs_bellman_ford_path_length(Q)):
    start = lst[0]
    targets = lst[1]
    
    for end in targets.keys():
        if end!=start:
            try:
                Q.get_edge_data(start, end)['weight'] = targets[end]
            except:
                nx.set_edge_attributes(Q, {(start, end):targets[end]}, 'weight')
                
                
# randomly drop edges from the graph

In [ ]:
for e in Q.edges():
    weight = Q.get_edge_data(e[0], e[1])['weight']
    if np.random.randn(0, 1)<0.8 and e not in nx.minimum_cut:
        Q.remove(e[0], e[1])
        
    if not nx.is_connected(Q):
        Q.add_edge(e[0], e[1], weight=weight)

In [ ]:
draw_graph(Q)

In [ ]:
start_index = np.random.choice(np.arange(0, 10))
start_index

In [ ]:
# randomly assign vertex weights
adj_matrix = nx.adj_matrix(Q).todense()
vertex_weights = np.random.randint(1, 10, len(adj_matrix))
adj_matrix[np.arange(len(adj_matrix)), np.arange(len(adj_matrix))] = vertex_weights
adj_matrix

In [ ]:
tour, conquer = solver(Q, 0, adj_matrix.tolist())
tour, conquer

In [ ]:
calculate_cost(tour, conquer, Q)

In [ ]:
draw_graph(Q)

In [ ]:
for i in range(len(Q.nodes)):
    print("Conquering cost {}: {}".format(i, Q.node[i]['conquering_cost']))

### Generate a new tree

In [ ]:
# now try to generate a random tree
from networkx.generators.trees import random_tree

T = random_tree(10)
random_weights_edge = np.random.randint(1, 10, len(T.edges()))
random_weights_vertex = np.random.randint(1, 10, len(T.nodes()))

nx.set_edge_attributes(T, dict(zip(T.edges, random_weights_edge)), 'weight')

In [ ]:
adj_matrix = nx.adj_matrix(T).todense()
adj_matrix[range(len(adj_matrix)), range(len(adj_matrix))] = random_weights_vertex
adj_matrix

In [ ]:
solver(T, start_index, adj_matrix.tolist())

In [ ]:
print("Start Index {}".format(start_index))

tour, conquer = solver(T, start_index, adj_matrix.tolist())

for i in range(len(T.nodes)):
    print("Conquering cost {}: {}".format(i, T.node[i]['conquering_cost']))

print("tour: {}".format(tour))
print("conquer: {}".format(conquer))
print("Total Cost {}".format(calculate_cost(tour, conquer, T)))
draw_graph(T)

Algo not so good in this case, need to find a good algo for weighted dominating set on tree.
A takeway is that in the case of very sparse stuff we don't need heuristics but more precise choice of vertices.
There actually exists a DP solution to weighted vertex cover for Trees.
I can check whether it is near-tree through the # of edges.

### Sparse graph oriented algorithm

For trees we do have dp algorithms that can solve min edge, vertex dominating set. If you have something that is very loosely connected you can try removing very heavy edges until it is a tree and then potentially run that algorithm on it. 

Another option would be to find an mst (isn't that what the above thing is tho) and then run the algo on it. 

I get it but implementation is a little tricky http://courses.csail.mit.edu/6.006/fall09/lecture_notes/lecture21.pdf

In [ ]:
def sparse_graph_algo(T, start_index):
    """OMG what the fuck did I just code up. AAAAARGH"""
    
    """
    This is a DP algorithm that given a tree calculate sthe 
    There are two cases:

    a) Root belongs to the vertex cover or it doesn't. In this case do 1 + size of left + size of right

    DP(v) = min. dom. set for subtree rooted at v
    DP'(v) = min. dom. set for subtree rooted at v not requiring v

    recursion

    DP(v) = min{1 + sum(DP'(c) for c in children[v]), # YES, DOMINATE the root v
                min_children{1 + sum(DP'(g) for g in children(d)) + sum(DP[c] for c new d in children[v])}}

    DP'(v) = min{1 + sum(DP'(c) for children[v]),
                 sum(DP[c] for c in children[v])}


    # I guess build everything up in post order
    """
    
    root = find_leaves(T)[0]

    T_prime = dfs_tree(T, root)

    post_order = list(dfs_postorder_nodes(T, root)) # using post order to traverse the nodes so leaves are up first


    dp = np.zeros(len(T.nodes))-1
    dp_prime = np.zeros(len(T.nodes))-1

    def is_leaf(node, T):
        return T.out_degree(node)==0 and T.in_degree(node)==1

    # establish the based cases for the two problems for the leaf nodes
    for ix in range(len(post_order)):
        node = post_order[ix]

        if is_leaf(node, T_prime):
            dp[ix] = 1 # start it easy and then change to accomodate weights
            dp_prime[ix] = 0

    # now let's build from the bottom up

    conquer = []

    for ix in range(len(post_order)):
        if dp[ix]!=-1:
            pass
        else:
            v = post_order[ix]
            #print(ix, v)
            children = list(T_prime.neighbors(v))
            children_ix = [post_order.index(c) for c in children]
            #print(children)

            ####### DP(v) UPDATE #########
            cost = T.node[v]['conquering_cost']

            if is_leaf(v, T_prime):
                parent = list(T_prime.predecessors(v))[0]
                cost += 2*T.get_edge_data(parent, v)['weight']

            # left part
            left = cost + sum([dp_prime[i] for i in children_ix]) #1 + sum(DP'(c) for c in children[v])

            # right part
            vals = []

            for d in children_ix:
                children2 = list(T_prime.neighbors(post_order[d]))
                children_ix2 = [post_order.index(c) for c in children2]
                cost2 = T.node[post_order[d]]['conquering_cost']

                if is_leaf(d, T_prime):
                    parent2 = list(T_prime.predecessors(d))[0]
                    cost2 += 2*T.get_edge_data(parent2, d)['weight']

                #print(c)
                #print(sum([dp[c] for i in children_ix if i!= d]))
                vals.append(cost2 + sum([dp_prime[i] for i in children_ix2]) + sum([dp[i] for i in children_ix if i!= d]))
            
            #print(vals)
            right = min(vals)

            dp[ix] = min(left, right)

            if left<right:
                conquer.append(post_order[ix])
                #print(post_order[ix])

            ####### DP'(v) UPDATE #########

            dp_prime[ix] = min(cost + sum([dp_prime[c] for c in children_ix]), sum([dp[c] for c in children_ix]))

            # store the result
            if dp_prime[ix] == cost + sum([dp_prime[c] for c in children_ix]):
                conquer.append(post_order[ix])
                #print(post_order[ix])

    conquer = list(set(conquer))
    
    # need to find a way to do the tour on the tree idk why but find_traversal does not work
    ST = steinertree.steiner_tree(T, conquer+[start_index])
    traversal = find_traversal(ST, start_index)
    return traversal, conquer

t, c = sparse_graph_algo(T, 8)
calculate_cost(t, c, T)

In [ ]:
# find mst first and the traversal, then decide efficiently which ones to conquer
from networkx.algorithms.tree.mst import minimum_spanning_tree

new= minimum_spanning_tree(T, weight="weight").nodes()

In [ ]:
draw_graph(T)

In [ ]:
# implementation of a min vertex cover. 
def find_leaves(G):
    leaves = []
    for n in G.nodes():
        if G.degree(n)==1:
            leaves.append(n)
    return leaves


def find_leaves_with_parent(parent, leaves, G):
    lst = []
    
    for leaf in leaves:
        if parent == list(T_prime.neighbors(leaf))[0]:
            lst.append(leaf)
    return lst



# testing tree
# tree = nx.Graph()
# tree.add_node(1)
# tree.add_node(2)
# tree.add_node(3)
# tree.add_node(4)

# tree.add_edge(1,2)
# tree.add_edge(1,3)
# tree.add_edge(2,4)
# tree.add_edge(2,5)
# tree.add_edge(5, 6)

# nx.draw(tree, with_labels=True)